In [1]:
import histogram_manager as hm
import ROOT
import utils
import stage1
import numpy as np

%jsroot on

Welcome to JupyROOT 6.28/00


In [2]:
hists =  hm.HistogramManager("../output/2023apr_asym_B_constbin.root")
hists

<HistogramManager(filename='../output/2023apr_asym_B_constbin.root', total_objects=353)>


Name              | Type  
---------------------------
hEgam             | 📂 TH1D
hEgam_gate45      | 📂 TH1D
hEn               | 📂 TH1D
hEn_gate9394      | 📂 TH1D
hEn_gate9394SE    | 📂 TH1D
hEn_gate9394SEbkg | 📂 TH1D
hEn_gate9394bkg   | 📂 TH1D
hEn_gtzero        | 📂 TH1D
hEn_zero          | 📂 TH1D
hPulseHeight      | 📂 TH1D
hTOF_mus          | 📂 TH1D
hEn_gate477_all   | 📊 TH1D

In [3]:
hEn_gate477_all = hists.get("hEn_gate477_all")

c = ROOT.TCanvas()
hEn_gate477_all.Draw()
hEn_gate477_all.GetXaxis().SetRangeUser(1,100)
c.Draw()

In [4]:
bin_edges_array = hEn_gate477_all.GetXaxis().GetXbins()
xbins_En = np.array([bin_edges_array[i] for i in range(bin_edges_array.GetSize())])


# down = 0
# up =1000000
# Nbins = 1000000

# xbins_En1 = utils.get_xbins_En_10ns(Nbins, down, up, 21.5)

In [5]:
c = ROOT.TCanvas()
hEn_sig_tot_10B = stage1.get_cross_section(stage1.CROSS_SECTION_PATH, xbins_En)
hEn_sig_tot_10B.Draw()
hEn_sig_tot_10B.GetXaxis().SetRangeUser(1,100)
c.Draw()

In [6]:
n = 1.41E23 # number density of 10Co[atom/cm^3]
d = 0.1 # target thickness [cm]
f = 1E-24


hEn_divisor = hEn_gate477_all.Clone()
hEn_divisor.SetTitle("hEn_divisor")
hEn_divisor.SetName("hEn_divisor")


for bin in range(1, hEn_divisor.GetNbinsX() + 1):  
    original_value = hEn_divisor.GetBinContent(bin)
    hEn_divisor.SetBinContent(bin, (1 - ROOT.TMath.Exp(-1 * n * d * f * original_value)))


c = ROOT.TCanvas()
hEn_divisor.GetXaxis().SetRangeUser(1,100)
hEn_divisor.Draw()
c.Draw()

In [7]:
# hEn_beam_intensity = hEn_gate477_all.Clone()

hEn_beam_intensity = hEn_gate477_all/hEn_divisor


hEn_beam_intensity.Draw()
hEn_beam_intensity.GetXaxis().SetRangeUser(1,100)
hEn_beam_intensity.SetName("hEn_beam_intensity")
hEn_beam_intensity.SetTitle("hEn_beam_intensity")
hEn_beam_intensity.GetYaxis().SetTitle("Intensity [A.U.]")

x_point = 4.5
bin_index = hEn_beam_intensity.FindBin(x_point)
entry = hEn_beam_intensity.GetBinContent(bin_index)
# print(entry)
hEn_beam_intensity.Scale(1/entry)


c = ROOT.TCanvas()
hEn_beam_intensity.Draw()
c.Draw()

In [8]:
beam_hist_dict = {"hEn_gate477_all":hEn_gate477_all,
                  "hEn_sig_tot_10B":hEn_sig_tot_10B,
                  "hEn_divisor":hEn_divisor,
                  "hEn_beam_intensity":hEn_beam_intensity}

beam_hists = hm.HistogramManager(beam_hist_dict)
beam_hists.write("../output/2023apr_asym_beam_constbin.root")